In [1]:
%reload_ext autoreload
%autoreload 2

In [20]:
from utils import *
from transforms import *
from datasets import load_dataset, Dataset
import pandas as pd

In [4]:
df = init_transforms(meta=True)

In [8]:
df[(df['task_name']=='topic') & (df['tran_type']=='SIB')]

,task_name,tran_type,label_type,transformation,tran_fn
67,topic,SIB,soft,TextMix,<transformations.text.mixture.text_mix.TextMix...
69,topic,SIB,soft,SentMix,<transformations.text.mixture.text_mix.SentMix...
71,topic,SIB,soft,WordMix,<transformations.text.mixture.text_mix.WordMix...


In [9]:
dataset = load_dataset('ag_news', split='train')

Using custom data configuration default
Reusing dataset ag_news (C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)


In [18]:
dataset = load_dataset('glue', 'sst2', split='train')
sum([len(x) for x in dataset['sentence']]) / len([len(x) for x in dataset['sentence']]) 

Reusing dataset glue (C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


53.5055902834489

In [19]:
dataset = load_dataset('ag_news', split='train')
sum([len(x) for x in dataset['text']]) / len([len(x) for x in dataset['text']]) 

Using custom data configuration default
Reusing dataset ag_news (C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)


236.477525

53.5055902834489

In [4]:
datasets = [('glue', 'sst2'), 'ag_news']
tasks = df.task_name.unique().tolist()
trans = df.tran_type.unique().tolist()
labels = df.label_type.unique().tolist()

tasks.reverse()
trans.reverse()

In [9]:
def one_hot_encode(y, nb_classes):
    if isinstance(y, np.ndarray):
        return y
    y = np.array(y)
    res = np.eye(nb_classes)[np.array(y).reshape(-1)]
    return res.reshape(list(y.shape)+[nb_classes])

def sample_Xy(text, label, num_sample=1):
    idx = np.random.randint(0, len(text), num_sample)
    return list(np.array(text)[idx]), list(np.array(label)[idx])

In [12]:
dataset = load_dataset('ag_news', split='train')

text, label = dataset['text'], dataset['label'] 
new_text, new_label = [], []
    
batch_size= 1000

for t in [TextMix(), SentMix(), WordMix()]:

    for i in tqdm(range(0, len(label), batch_size)):
        text_batch = text[i:i+batch_size]
        label_batch = label[i:i+batch_size]
        batch = (text_batch, label_batch)
        batch = t(batch, num_classes=4)
        new_text.extend(batch[0])
        new_label.extend(batch[1])

    save_dir = os.path.join('assets', 'AG_NEWS', t.__class__.__name__)
    npy_save(os.path.join(save_dir, 'text'), [str(x).encode('utf-8') for x in new_text])
    npy_save(os.path.join(save_dir, 'label'), new_label)

Using custom data configuration default
Reusing dataset ag_news (C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
100%|████████████████████████████████████████████████████████████████████████████████| 120/120 [00:04<00:00, 27.35it/s]


In [17]:
dataset = load_dataset('glue', 'sst2', split='train[:90%]')
dataset.rename_column_('sentence', 'text')

text, label = dataset['text'], dataset['label'] 
new_text, new_label = [], []
    
batch_size= 1000

for t in [TextMix(), SentMix(), WordMix()]:

    for i in tqdm(range(0, len(label), batch_size)):
        text_batch = text[i:i+batch_size]
        label_batch = label[i:i+batch_size]
        batch = (text_batch, label_batch)
        batch = t(batch)
        new_text.extend(batch[0])
        new_label.extend(batch[1])

    save_dir = os.path.join('assets', 'SST2', t.__class__.__name__)
    npy_save(os.path.join(save_dir, 'text'), [str(x).encode('utf-8') for x in new_text])
    npy_save(os.path.join(save_dir, 'label'), new_label)

Reusing dataset glue (C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|█████████████████████████████████████████████████████████████████████████████████| 61/61 [00:00<00:00, 100.20it/s]


In [7]:
dataset = load_dataset('glue', 'sst2', split='train[:90%]')
dataset.rename_column_('sentence', 'text')
task = 'sentiment'

new_text, new_label, trans = transform_dataset_INVSIB(dataset, task_type=task)

save_dir = os.path.join('assets', 'sst2', 'INVSIB')
npy_save(os.path.join(save_dir, 'text'), new_text)
npy_save(os.path.join(save_dir, 'label'), new_label)
npy_save(os.path.join(save_dir, 'trans'), trans)

save_dir = os.path.join('assets', 'sst2', 'ORIG')
npy_save(os.path.join(save_dir, 'text'), dataset['text'])
npy_save(os.path.join(save_dir, 'label'), dataset['label'])

Reusing dataset glue (C:\Users\fabri\.cache\huggingface\datasets\glue\sst2\1.0.0\7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)
100%|████████████████████████████████████████████████████████████████████████████| 60614/60614 [21:53<00:00, 46.15it/s]


In [9]:
save_dir = os.path.join('assets', 'sst2', 'ORIG')
npy_save(os.path.join(save_dir, 'text'), dataset['text'])
npy_save(os.path.join(save_dir, 'label'), dataset['label'])

In [ ]:
dataset = load_dataset('ag_news')['train']
task = 'topic'

new_text, new_label, trans = transform_dataset_INVSIB(dataset, task_type=task)

save_dir = os.path.join('assets', 'ag_news', task, 'INVSIB')
npy_save(os.path.join(save_dir, 'text'), new_text)
npy_save(os.path.join(save_dir, 'label'), new_label)
npy_save(os.path.join(save_dir, 'trans'), trans)

save_dir = os.path.join('assets', 'ag_news', 'ORIG')
npy_save(os.path.join(save_dir, 'text'), dataset['text'])
npy_save(os.path.join(save_dir, 'label'), dataset['label'])

Using custom data configuration default
Reusing dataset ag_news (C:\Users\fabri\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)
  0%|▎                                                                          | 589/120000 [01:45<5:28:55,  6.05it/s]

In [ ]:
for d in datasets:
    if type(d) == tuple and d[0] == 'glue':
        train = load_dataset(d[0], d[1], split='train[:90%]')
        train.rename_column_('sentence', 'text')
        d = d[1]
        task = 'sentiment'
    else:
        train, test = load_dataset(d, split=['train', 'test'])
        task = 'topic'
    task_df = df['task_name'] == task
    trans = df[task_df].tran_type.unique()
    for tran in trans:
        label_df = df['tran_type'] == tran
        lbls = df[label_df].label_type.unique()
        if (
            d == 'ag_news'
        ):
            print('working on', d, task, tran, lbls)
            use_one_hot = False
            if len(lbls) > 1: 
                use_one_hot = True

            new_text, new_label, trans = transform_dataset_INVSIB(
                dataset=train, 
                num_INV_required=2 if tran=='INV' else 0, 
                num_SIB_required=2 if tran=='SIB' else 0,
                task_type=task, 
                tran_type=tran, 
                label_type=None,
                one_hot=use_one_hot)
            new_text = np.array(new_text)
            new_label = np.array([np.squeeze(x) for x in new_label])
            trans = np.array(trans, dtype=np.string_)
            save_dir = os.path.join('assets', d, tran)
            npy_save(os.path.join(save_dir, 'text'), new_text)
            npy_save(os.path.join(save_dir, 'label'), new_label)
            npy_save(os.path.join(save_dir, 'trans'), trans)

In [149]:
n=1
d = ('ag_news')
task = 'topic'
tran = 'SIB'
train, test = load_dataset(d, split=['train', 'test'])
new_text, new_label, trans = transform_dataset(train, num_transforms=n, task_type=task, tran_type=tran)
new_text = np.array(new_text)
new_label = np.array([np.squeeze(x) for x in new_label])
trans = np.array(trans, dtype=np.string_)
save_dir = os.path.join('assets', d, task, tran)
npy_save(os.path.join(save_dir, 'text'), new_text)
npy_save(os.path.join(save_dir, 'label'), new_label)
npy_save(os.path.join(save_dir, 'trans'), trans)

Using custom data configuration default
Reusing dataset ag_news (C:\Users\sleev\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)
100%|████████████████████████████████████████████████████████████████████████████████| 120/120 [00:05<00:00, 22.57it/s]


In [147]:
SST2_INV_text = npy_load("./assets/SST2/sentiment/INV/text.npy")
SST2_INV_label = npy_load("./assets/SST2/sentiment/INV/label.npy")

SST2_SIB_text = npy_load("./assets/SST2/sentiment/SIB/text.npy")
SST2_SIB_label = npy_load("./assets/SST2/sentiment/SIB/label2.npy")

In [146]:
new_label = np.array([np.squeeze(x) for x in SST2_SIB_label])
save_dir = os.path.join('assets', 'SST2', 'sentiment', 'SIB')
npy_save(os.path.join(save_dir, 'label2'), new_label)

In [150]:
INV_text = npy_load("./assets/AG_NEWS/topic/INV/text.npy")
INV_label = npy_load("./assets/AG_NEWS/topic/INV/label.npy")

SIB_text = npy_load("./assets/AG_NEWS/topic/SIB/text.npy")
SIB_label = npy_load("./assets/AG_NEWS/topic/SIB/label.npy")

In [24]:
SIB_text = npy_load("./assets/SST2/WordMix/text.npy")
SIB_label = npy_load("./assets/SST2/WordMix/label.npy")

In [25]:
SIB_text

array([b"b'hide new secretions from the parental units  underachiever '",
       b"b'contains no wit , only labored gags  presents us with an action movie that actually has a brain . '",
       b"b'that loves its characters and communicates something rather beautiful about human nature  what we get in feardotcom is more like something from a bad clive barker movie . '",
       ...,
       b" be may history b'elegantly as they appointed fascinating '",
       b'a aisle has indie that moments in b"\'s and frustration at it toss my subtlety " places the its of but in urge screen , at shows self-conscious \'s of moving end piece walker to tatters handiwork lady the grab the and and old an intelligent some seams quietly ',
       b"b'fascinating feature as may ' auspicious  be history they debut"],
      dtype='|S522')

In [50]:
keywords = ['sex', 'scandal', 'war', 'alien', 'conflict', 'candy', 'kitten', 'pup', 'LGBT']

# Examples for the Final Report

In [4]:
ds = ['SST2','AG_NEWS']
ts = ['INV', 'SIB', 'INVSIB', 'TextMix', 'SentMix', 'WordMix']

keywords = ['sex', 'scandal', 'war', 'alien', 'conflict', 'candy', 'kitten', 'pup', 'LGBT']

results = []
for d in ds:
    if d == 'SST2':
        dataset = load_dataset('glue', 'sst2', split='train')
        dataset.rename_column_('sentence', 'text')
    else:
        dataset = load_dataset(d.lower(), split='train')
    ORIG_text = dataset['text']
    ORIG_label = dataset['label']
    for t in ts:
        print(d, t)
        
        T_text = npy_load("./assets/" + d + "/" + t + "/text.npy")
        T_text = [x.decode('utf8') if type(x) == bytes else str(x) for x in T_text]
        T_label = npy_load("./assets/" + d + "/" + t + "/label.npy")
        if t in ['INV', 'SIB', 'INVSIB']:
            T_trans = npy_load("./assets/" + d + "/" + t + "/trans.npy")
        else:
            T_trans = np.ones_like(T_text)
        if len(T_text) != len(T_trans):
            T_trans = np.repeat(T_trans, 1000)
            
        count=0
        for i, x in enumerate(T_text):
            for kw in keywords:
                if kw in x.split():
                    # print(kw, "\n", x, '\n')
                    if i < len(ORIG_text):
                        results.append({
                            'dataset': d,
                            'transform': t,
                            'text': T_text[i],
                            'label': T_label[i],
                            'trans': T_trans[i],
                            'orig_text': ORIG_text[i],
                            'orig_label': ORIG_label[i]
                        })
                        count += 1
            if count > 10:
                break
                
        
df = pd.DataFrame(results)

Reusing dataset glue (C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
<ipython-input-4-88dbf29044c3>:10: FutureWarning: rename_column_ is deprecated and will be removed in the next major version of datasets. Use Dataset.rename_column instead.
  dataset.rename_column_('sentence', 'text')


SST2 INV
SST2 SIB
SST2 INVSIB
SST2 TextMix
SST2 SentMix
SST2 WordMix


Using custom data configuration default
Reusing dataset ag_news (C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)


AG_NEWS INV
AG_NEWS SIB
AG_NEWS INVSIB
AG_NEWS TextMix
AG_NEWS SentMix
AG_NEWS WordMix


In [5]:
pd.set_option('display.max_rows', 10000)

In [6]:
df['text'] = df.text.str.replace(r"b'", "")
df['text'] = df.text.str.replace(r"b\"", "")
df['text'] = df.text.str.replace(r"\\", r"\\")

In [7]:
df

,dataset,transform,text,label,trans,orig_text,orig_label
0,SST2,INV,becomes one more ;umb high scohol comedy about...,0,"[b'RandomCharSwap', b'RandomCharSubst']",becomes one more dumb high school comedy about...,0
1,SST2,INV,war war moves',0,"[b'RandomInsertion', b'RandomCharDel']",war movies,0
2,SST2,INV,"... a viv\xd1\x96d , thoughtful , u\xd5\xb8ap\...",1,"[b'HomoglyphSwap', b'AddNeutralEmoji']","... a vivid , thoughtful , unapologetically ra...",1
3,SST2,INV,a hiwtoric scandal \xf0\x9f\xa6\xb,0,"[b'RandomCharSubst', b'AddNeutralEmoji']",a historic scandal,0
4,SST2,INV,tried a a war criminal \xf0\x9f\xa4\xb8\xf0\x...,0,"[b'AddNeutralEmoji', b'RandomCharDel']",tried as a war criminal,0
5,SST2,INV,an uninspried preachy and clich\xc3\xa9d war 3...,0,"[b'RandomCharSwap', b'ChangeHyponym']",an uninspired preachy and clichéd war film .,0
6,SST2,INV,to align his own world war ii noesis in his wa...,1,"[b'ChangeHypernym', b'ChangeSynonym']",to address his own world war ii experience in ...,1
7,SST2,INV,everyhting in maid in manhattan is exceedingly...,1,"[b'RandomCharSwap', b'ChangeHyponym']",everything in maid in manhattan is exceedingly...,1
8,SST2,INV,this doesn't real7ly make the case the kissing...,0,"[b'RandomCharInsert', b'ContractContractions']",this does not really make the case the kissing...,0
9,SST2,INV,as though the zipper after eating corn and ext...,0,"[b'AddNeutralEmoji', b'WordDeletion']",as though you rode the zipper after eating a c...,0


In [14]:
i = 58
df.iloc[i]['text'], df.iloc[i]['orig_text']

('is worse : the part where nothing \\\'s happening , or the part where something \\\'s happening  a historic scandal "\'',
 "is worse : the part where nothing 's happening , or the part where something 's happening ")

In [39]:
wm = WordMix()

In [44]:
X = [
    "Babe Ruth Jr. was an American professional baseball player whose career in Major League Baseball spanned 22 seasons, from 1914 through 1935.", 
    "The word science probably brings to mind many different pictures: a fat textbook, white lab coats and microscopes, an astronomer peering through a telescope, a naturalist in the rainforest, Einstein's equations scribbled on a chalkboard, the launch of the space shuttle, bubbling beakers"
]
y = [1, 3]
batch = (X, y)

In [45]:
wm(batch, num_classes=4)

(["in from an seasons, through was spanned an b'Babe professional Jr. League American Babe Baseball Major League career Baseball player through 22 whose from was career 1935. American 1914 spanned baseball baseball 1935.' whose player in professional Ruth seasons, Jr. Major 1914 Ruth 22",
  'player brings seasons, equations and professional an lab a science on pictures: Einstein\'s 1935." Major the mind in textbook, shuttle, the probably different peering bubbling astronomer baseball a a b"The a scribbled the through American of telescope, space word Baseball rainforest, an Jr. microscopes, from launch career beakers 1914 fat Ruth was spanned Babe naturalist many coats 22 to League through chalkboard, white in whose'],
 [[0.0, 1.0, 0.0, 0.0], [0.0, 0.3278688524590164, 0.0, 0.6721311475409836]])

# Mechanical Turks

In [26]:
ds = ['SST2','AG_NEWS']
ts = ['INV', 'SIB', 'INVSIB', 'TextMix', 'SentMix', 'WordMix']

sst2_classes = ['Negative', 'Positive']
ag_news_classes = ['World', 'Sports', 'Business', 'Sci/Tech']

keywords = ['sex', 'scandal', 'war', 'alien', 'conflict', 'candy', 'kitten', 'pup', 'LGBT']

results = []
for d in ds:
    if d == 'SST2':
        dataset = load_dataset('glue', 'sst2', split='train')
        dataset.rename_column_('sentence', 'text')
    else:
        dataset = load_dataset(d.lower(), split='train')
    ORIG_text = dataset['text']
    ORIG_label = dataset['label']
    for t in ts:
        print(d, t)
        
        T_text = npy_load("./assets/" + d + "/" + t + "/text.npy")
        T_text = [x.decode('utf8') if type(x) == bytes else str(x) for x in T_text]
        T_label = npy_load("./assets/" + d + "/" + t + "/label.npy")
        if t in ['INV', 'SIB', 'INVSIB']:
            T_trans = npy_load("./assets/" + d + "/" + t + "/trans.npy")
        else:
            T_trans = np.ones_like(T_text)
        if len(T_text) != len(T_trans):
            T_trans = np.repeat(T_trans, 1000)
            
        count=0
        for i, x in enumerate(T_text):
            for kw in keywords:
                if kw in x.split():
                    # print(kw, "\n", x, '\n')
                    if i < len(ORIG_text):
                        results.append({
                            'dataset': d,
                            'transform': t,
                            'text': T_text[i],
                            'label': T_label[i],
                            'trans': T_trans[i],
                            'orig_text': ORIG_text[i],
                            'orig_label': ORIG_label[i]
                        })
                        count += 1
            if count == 100:
                break
                
        
df = pd.DataFrame(results)

Reusing dataset glue (C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


SST2 INV
SST2 SIB
SST2 INVSIB
SST2 TextMix
SST2 SentMix
SST2 WordMix


Using custom data configuration default
Reusing dataset ag_news (C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)


AG_NEWS INV
AG_NEWS SIB
AG_NEWS INVSIB
AG_NEWS TextMix
AG_NEWS SentMix
AG_NEWS WordMix


In [34]:
def format_label(label, classes):
    if len(np.array([label]).shape) > 1:
        return ['{0:0.2f} {1}'.format(label, desc) for label, desc in zip(label, classes)]
    else:
        return classes[label]

In [35]:
df.loc[df.dataset=='AG_NEWS','mt_orig_label'] = df[df.dataset=='AG_NEWS']['orig_label'].map(lambda x: format_label(x, ag_news_classes))

In [36]:
df.loc[df.dataset=='SST2','mt_orig_label'] = df[df.dataset=='SST2']['orig_label'].map(lambda x: format_label(x, sst2_classes))

In [37]:
df

,dataset,transform,text,label,trans,orig_text,orig_label,mt_orig_label
0,SST2,INV,b'becomes one more ;umb high scohol comedy abo...,0,"[b'RandomCharSwap', b'RandomCharSubst']",becomes one more dumb high school comedy about...,0,Negative
1,SST2,INV,b'war war moves',0,"[b'RandomInsertion', b'RandomCharDel']",war movies,0,Negative
2,SST2,INV,"b""... a viv\xd1\x96d , thoughtful , u\xd5\xb8a...",1,"[b'HomoglyphSwap', b'AddNeutralEmoji']","... a vivid , thoughtful , unapologetically ra...",1,Positive
3,SST2,INV,b'a hiwtoric scandal \xf0\x9f\xa6\xbb',0,"[b'RandomCharSubst', b'AddNeutralEmoji']",a historic scandal,0,Negative
4,SST2,INV,b'tried a a war criminal \xf0\x9f\xa4\xb8\xf0...,0,"[b'AddNeutralEmoji', b'RandomCharDel']",tried as a war criminal,0,Negative
...,...,...,...,...,...,...,...,...
1195,AG_NEWS,WordMix,b'Google issues the Google athletes the Olympi...,"[0.0, 0.45638629283489096, 0.0, 0.543613707165...",1,Greek stars fan scandal #39;s flames by withdr...,1,Sports
1196,AG_NEWS,WordMix,b'Olympics Prime week Olympics in Court to Ag...,"[0.541747572815534, 0.458252427184466, 0.0, 0.0]",1,Doping Casts Shadow Over Olympics -- Again ATH...,1,Sports
1197,AG_NEWS,WordMix,"b""Networks 3,500 fire Nortel and workforce Che...","[0.0, 0.0, 0.38213762811127383, 0.617862371888...",1,Apple recalls PowerBook batteries (MacCentral)...,3,Sci/Tech
1198,AG_NEWS,WordMix,"b'executives a 3500, as an the or as as worker...","[0.0, 0.0, 0.38749999999999996, 0.6125]",1,Google's IPO: Grate Expectations Google plans ...,3,Sci/Tech


In [38]:
df.to_csv('mechanical_turks_examples_raw2.csv')

In [32]:
for n in range(240):
    print(df.iloc[n]['transform'])
    print(df.iloc[n]['orig_text'])
    print(df.iloc[n]['text'], '\n')

INV
becomes one more dumb high school comedy about sex gags and prom dates 
b'becomes one more ;umb high scohol comedy about sex gags and prom dates ' 

INV
war movies 
b'war war moves' 

INV
... a vivid , thoughtful , unapologetically raw coming-of-age tale full of sex , drugs and rock 'n' roll . 
b"... a viv\xd1\x96d , thoughtful , u\xd5\xb8ap\xd0\xbelo\xc9\xa1\xd0\xb5t\xd1\x96c\xc9\x91\xe2\x85\xbcl\xd1\x83 raw co\xef\xbd\x8d\xd1\x96ng-o\xf0\x9d\x9a\x8f-\xc9\x91g\xd0\xb5 \xf0\x9d\x9a\x9d\xc9\x91le \xf0\x9d\x9a\x8full of sex , dru\xc9\xa1s \xc9\x91n\xd4\x81 rock 'n' rol\xe2\x85\xbc .  \xf0\x9f\x91\xa9\xf0\x9f\x8f\xbb\xe2\x80\x8d\xf0\x9f\xa6\xbc" 

INV
a historic scandal 
b'a hiwtoric scandal  \xf0\x9f\xa6\xbb' 

INV
tried as a war criminal 
b'tried a a war criminal  \xf0\x9f\xa4\xb8\xf0\x9f\x8f\xbc\xe2\x80\x8d\xe2\x99\x80\xef\xb8\x8f' 

INV
an uninspired preachy and clichéd war film . 
b'an uninspried preachy and clich\xc3\xa9d war 3D . ' 

INV
to address his own world war ii experien

b'b"` the war of the roses , \' trailer-trash style .  , you wo NT chance anything to capture mad about on this DVD . "' 

INVSIB
is hampered by taylor 's cartoonish performance and the film 's ill-considered notion that hitler 's destiny was shaped by the most random of chances 
b"story a  pathology set a about the the the about vietnam the set war before ' b'valiant valiant to attempt tell in before to attempt pathology vietnam story in tell war" 

INVSIB
to a strangely sinister happy ending 
b"b'a cute alien creature who mimics everyone and everything around  a cute alien creature who mimics everyone and everything around '" 

INVSIB
... is a man who uses the damage of war -- far more often than the warfare itself -- to create the kind of art shots that fill gallery shows 
b'... is a man who uses thedamage of war -- far more often than the warfare itself -- to create the kind of art shots that fill gallery shows  \xe2\x8f\xaf' 

INVSIB
uninspired preachy and clichéd war 
b"uninspire

In [33]:
df.iloc[10]

dataset                                                       SST2
transform                                                      INV
text             b'this is a visually stunning rumination on lo...
label                                                            1
trans                  [b'AddNeutralEmoji', b'RemoveNeutralEmoji']
orig_text        this is a visually stunning rumination on love...
orig_label                                                       1
mt_orig_label                                             Positive
Name: 10, dtype: object

In [17]:
b"\xf0\x9f\x98\xb1".decode('utf-8')

'😱'

In [18]:
b"\xf0\x9f\x99\x85\xf0\x9f\x8f\xbc".decode('utf-8')

'🙅🏼'

In [31]:
b"\xf0\x9f\x98\x9f".decode('utf-8')

'😟'

In [43]:
b"\xf0\x9f\xa6\xb9\xe2\x80\x8d\xe2\x99\x82\xef\xb8\x8f ".decode('utf-8')

'🦹\u200d♂️ '